<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/SEC_LLM_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import google.generativeai as genai
import time
import json
from google.colab import userdata

# --- API Key Setup (as provided by you, directly used) ---
GOOGLE_API_KEY = userdata.get('GEMINI')
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google Generative AI configured successfully using Colab Secrets.")
else:
    print("WARNING: GOOGLE_API_KEY not found in Colab Secrets. Please ensure 'GEMINI' secret is set.")
    print("API calls will likely fail. Proceeding with unconfigured API.")

# --- Agent Configuration ---
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash"

try:
    AGENTIC_MODEL = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
    print(f"Gemini model '{AgentConfig.LLM_MODEL_NAME}' initialized for agentic decisions.")
except Exception as e:
    print(f"ERROR: Failed to initialize Gemini model. Please check your API key and model name. Error: {e}")
    AGENTIC_MODEL = None

# --- Agent Functions ---
# Note: The LLM is used exclusively within the analysis_agent for this demo.

def analysis_agent(email_content):
    """
    The Analysis Agent, powered by a Gemini LLM, to analyze email content.
    It returns a structured JSON object for programmatic parsing.
    """
    if not AGENTIC_MODEL:
        print("Analysis Agent: ERROR - LLM model not available.")
        return {"error": "LLM not configured."}

    print("  -> Analysis Agent: Sending email to LLM for deep analysis...")

    # Prompt the LLM to act as a security analyst and return a structured JSON.
    # This is a core part of agentic design: giving the LLM a clear role and output format.
    prompt = f"""
    You are an expert cybersecurity analyst. Your task is to analyze the following email for signs of a phishing attempt.
    Based on your analysis, please return a JSON object with the following structure:
    {{
      "is_phishing": boolean,
      "suspicious_elements": string[],
      "threat_score": number,
      "recommended_action": string,
      "suspicious_url": string
    }}

    If the email is legitimate, "is_phishing" should be false and "threat_score" should be low (e.g., 0.1).
    If it is a phishing attempt, "is_phishing" should be true and "threat_score" should be high (e.g., 0.9).
    The "recommended_action" should be a brief description of what to do (e.g., "Quarantine email and block URL.").

    Email to analyze:
    ---
    {email_content}
    ---
    """

    try:
        # Instruct the model to return a JSON object, which is critical for programmatic use.
        response = AGENTIC_MODEL.generate_content(
            prompt,
            generation_config={"response_mime_type": "application/json"}
        )

        # The LLM's response is a string. We must parse it.
        json_output = json.loads(response.text)
        print("  -> Analysis Agent: Analysis complete. Returning structured data.")
        return json_output

    except Exception as e:
        print(f"  -> Analysis Agent: ERROR - LLM call failed or response was invalid. Error: {e}")
        return {"error": str(e), "is_phishing": False, "threat_score": 0.0}

def search_agent(url):
    """(MOCK) Simulates the Search Agent's function, checking external threat intel."""
    print(f"  -> Search Agent: Querying threat intelligence for URL: {url}")
    time.sleep(1) # Simulate network latency
    if "suspicious-payment-link.com" in url:
        return {"threat_intelligence_findings": "URL flagged as a known phishing site.", "is_malicious": True}
    return {"threat_intelligence_findings": "No known threats found.", "is_malicious": False}

def action_agent(recommended_action, url_to_block):
    """(MOCK) Simulates the Action Agent's function."""
    print(f"  -> Action Agent: Taking action based on recommendation: '{recommended_action}'")
    if "Quarantine" in recommended_action:
        print(f"  -> Action Agent: Blocking URL '{url_to_block}' and quarantining email.")
        return "Action taken: Threat neutralized."
    return "Action taken: No action required."

def orchestrator_agent(email_text):
    """
    The Orchestrator Agent. It plans, delegates, and synthesizes the final response.
    """
    print("Orchestrator Agent: Received a new email to investigate.")

    # Step 1: Delegate the primary analysis to the LLM-powered Analysis Agent.
    analysis_results = analysis_agent(email_text)

    if analysis_results.get("error"):
        print(f"Orchestrator Agent: Investigation failed due to an error: {analysis_results['error']}")
        return "Investigation failed."

    # Step 2: Delegate external search if a suspicious URL was found.
    search_results = {}
    if analysis_results.get("suspicious_url"):
        search_results = search_agent(analysis_results["suspicious_url"])

    # Step 3: Synthesize findings and delegate action.
    print("\nOrchestrator Agent: All agents have reported. Synthesizing final report...")
    if analysis_results["is_phishing"] and search_results.get("is_malicious"):
        print(f"  - Final Conclusion: CRITICAL threat. LLM analysis and threat intelligence both confirm phishing.")
        action_agent(analysis_results["recommended_action"], analysis_results["suspicious_url"])
    else:
        print("  - Final Conclusion: Investigation complete. Threat level is low.")
        action_agent("None", None)

    print("\nOrchestrator Agent: Final workflow complete.\n" + "="*50 + "\n")

# --- Demo Usage ---

# Example 1: A legitimate email
legit_email = "Hello, your Q3 financial report is attached. Please review it at your convenience. Best regards, John from Accounting."
orchestrator_agent(legit_email)

# Example 2: A suspicious, phishing-like email
phishing_email = "Urgent action required! Your account has been compromised. Please update your payment information immediately via this link: https://suspicious-payment-link.com/login"
orchestrator_agent(phishing_email)

Google Generative AI configured successfully using Colab Secrets.
Gemini model 'gemini-2.5-flash' initialized for agentic decisions.
Orchestrator Agent: Received a new email to investigate.
  -> Analysis Agent: Sending email to LLM for deep analysis...
  -> Analysis Agent: Analysis complete. Returning structured data.

Orchestrator Agent: All agents have reported. Synthesizing final report...
  - Final Conclusion: Investigation complete. Threat level is low.
  -> Action Agent: Taking action based on recommendation: 'None'

Orchestrator Agent: Final workflow complete.

Orchestrator Agent: Received a new email to investigate.
  -> Analysis Agent: Sending email to LLM for deep analysis...
  -> Analysis Agent: Analysis complete. Returning structured data.
  -> Search Agent: Querying threat intelligence for URL: https://suspicious-payment-link.com/login

Orchestrator Agent: All agents have reported. Synthesizing final report...
  - Final Conclusion: CRITICAL threat. LLM analysis and threat 